In [4]:
import cohere
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

In [5]:
load_dotenv('.env')
# Initializes your app with your bot token and socket mode handler
cohere_api_key=os.environ.get("cohere_api_key")

In [6]:
#loader = UnstructuredPDFLoader("Light.pdf", extract_images=True)
loader = PyPDFLoader("Light.pdf", extract_images=True)
pages = loader.load_and_split()

In [7]:
len(pages)

44

In [8]:
pages[0].page_content

'Chapter One\nELECTRIC CHARGES\nAND FIELDS\n1.1  I NTRODUCTION\nAll of us have the experience of seeing a spark or hearing a crackle when\nwe take of f our synthetic clothes or sweater , particularly in dry weather .\nHave you ever tried to find any explanation for this phenomenon? Another\ncommon example of electric discharge is the lightning that we see in the\nsky during thunderstor ms. W e also experience a sensation of an electric\nshock either while opening the door of a car or holding the iron bar of a\nbus after sliding from our seat. The reason for these experiences is\ndischarge of electric charges through our body, which were accumulated\ndue to rubbing of insulating sur faces. Y ou might have also hear d that\nthis is due to generation of static electricity. This is precisely the topic we\nare going to discuss in this and the next chapter . Static means anything\nthat does not move or change with time. Electrostatics deals with\nthe study of forces, fields and potentials ar

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


r_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024 , 
    chunk_overlap = 256,
    length_function  = len
)

docs = r_splitter.split_documents(pages)
len(docs)

136

In [10]:
from langchain_cohere import CohereEmbeddings
embeddings_model = CohereEmbeddings(cohere_api_key=cohere_api_key)

In [11]:
persist_directory = 'embeddings/chroma_doc/'

In [12]:
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings_model,
    persist_directory=persist_directory
)

In [13]:
query = "What is COULOMB'S LAW?"
docs = vectordb.similarity_search(query, k=2)
print(docs[0].page_content)

with an identical uncharged sphere, the charge will spread over
the two spheres. By symmetry, the charge on each sphere will
be q/2*. Repeating this process, we can get charges q/2, q/4,
etc. Coulomb varied the distance for a fixed pair of charges and
measured the force for different separations. He then varied the
charges in pairs, keeping the distance fixed for each pair .
Comparing forces for different pairs of  charges at different
distances, Coulomb arrived at the relation, Eq. (1.1).
Coulomb’s law, a simple mathematical statement,  was
initially experimentally arrived at  in the manner described
above. While the original experiments established it at a
macroscopic scale, it has also been established down to
subatomic level  ( r ~ 10–10 m).
Coulomb discovered his law without knowing the explicit
magnitude of the charge. In fact, it is the other way round:
Coulomb’s law can now be employed to furnish a definition
for a unit of charge. In the relation, Eq. (1.1), k is so far


In [14]:
embedding_vector = embeddings_model.embed_query(query)
docs = vectordb.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

with an identical uncharged sphere, the charge will spread over
the two spheres. By symmetry, the charge on each sphere will
be q/2*. Repeating this process, we can get charges q/2, q/4,
etc. Coulomb varied the distance for a fixed pair of charges and
measured the force for different separations. He then varied the
charges in pairs, keeping the distance fixed for each pair .
Comparing forces for different pairs of  charges at different
distances, Coulomb arrived at the relation, Eq. (1.1).
Coulomb’s law, a simple mathematical statement,  was
initially experimentally arrived at  in the manner described
above. While the original experiments established it at a
macroscopic scale, it has also been established down to
subatomic level  ( r ~ 10–10 m).
Coulomb discovered his law without knowing the explicit
magnitude of the charge. In fact, it is the other way round:
Coulomb’s law can now be employed to furnish a definition
for a unit of charge. In the relation, Eq. (1.1), k is so far


In [15]:
from langchain_cohere import ChatCohere

In [16]:
from langchain.chains import ConversationalRetrievalChain, StuffDocumentsChain, create_history_aware_retriever, create_retrieval_chain, LLMChain 
from langchain import hub
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate


In [17]:
template = (
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "Follow up question: {question}"
)
prompt = PromptTemplate.from_template(template)
# combine_doc_chain = StuffDocumentsChain()
llm = ChatCohere(cohere_api_key=cohere_api_key , temperature=0)
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
retriever = vectordb.as_retriever()
# chain = ConversationalRetrievalChain(
#     retriever=retriever,
#     question_generator=question_generator_chain,
# )

C:\Users\dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [18]:
! pip install langchainhub

In [19]:
rephrase_prompt = hub.pull("langchain-ai/chat-langchain-rephrase")
llm = ChatCohere(cohere_api_key=cohere_api_key, temperature=0)
retriever = vectordb.as_retriever()
chat_retriever_chain = create_history_aware_retriever(
    llm = llm, retriever = retriever, prompt = rephrase_prompt
)

In [20]:
chat_retriever_chain.invoke({"input":"What is Quantisation of charge?"})

[Document(page_content='and conservation.\nQuantisation of electric charge means that total charge ( q) of a body\nis always an integral multiple of a basic quantum of charge ( e) i.e.,\nq = n e, where n = 0, ±1, ±2, ±3, .... Proton and electron have charges\n+e, –e, respectively. For macroscopic charges for which n is a very large\nnumber , quantisation of char ge can be ignor ed.\nAdditivity of electric charges means that the total charge of a system\nis the algebraic sum (i.e., the sum taking into account proper signs)\nof all individual charges in the system.\nConservation of electric charges means that the total charge of an\nisolated system remains unchanged with time. This means that when\n2024-25\n2024-25', metadata={'page': 36, 'source': 'Light.pdf'}),
 Document(page_content='and conservation.\nQuantisation of electric charge means that total charge ( q) of a body\nis always an integral multiple of a basic quantum of charge ( e) i.e.,\nq = n e, where n = 0, ±1, ±2, ±3, .... Pr

In [21]:
chat_retriever_chain.invoke({"input":"Give me specified and short answer."})

[Document(page_content='electricity through them. They are called insulators . Most  substances\nfall into one of the two classes stated above *.\nWhen some char ge is transferr ed to a conductor , it r eadily gets\ndistributed over the entir e sur face of the conductor . In contrast, if some\ncharge is put on an insulator , it stays at the same place. Y ou will lear n\nwhy this happens in the next chapter .\nThis property of the materials tells you why a nylon or plastic comb\ngets electrified on combing dry hair or on rubbing, but a metal article\n*There is a third category called semiconductors , which offer resistance to the\nmovement of charges which is intermediate between the conductors and\ninsulators.\n2024-25\n2024-25', metadata={'page': 2, 'source': 'Light.pdf'}),
 Document(page_content='electricity through them. They are called insulators . Most  substances\nfall into one of the two classes stated above *.\nWhen some char ge is transferr ed to a conductor , it r eadily gets

In [22]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms.cohere import Cohere

In [23]:
chain = load_qa_chain(Cohere(), chain_type="stuff", verbose=True)

C:\Users\dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


In [24]:
query1 =  "what is Quantisation?"
docs = vectordb.similarity_search(query1)
chain.run(input_documents = docs, question = query1)

C:\Users\dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

and conservation.
Quantisation of electric charge means that total charge ( q) of a body
is always an integral multiple of a basic quantum of charge ( e) i.e.,
q = n e, where n = 0, ±1, ±2, ±3, .... Proton and electron have charges
+e, –e, respectively. For macroscopic charges for which n is a very large
number , quantisation of char ge can be ignor ed.
Additivity of electric charges means that the total charge of a system
is the algebraic sum (i.e., the sum taking into account proper signs)
of all individual charges in the system.
Conservation of electric charges means that the total charge of an
isolated system remains unchanged with time. This means that when
2024-25
2024-25

and conservation.
Quantisation of electric

" Quantisation is the process of assigning specific discrete numerical values to certain physical quantities, such as electric charge. In the context of electric charge, quantisation means that the total charge of a body is always an integral multiple of a basic quantum of charge, denoted as e. This property arises from the nature of electromagnetic particles, such as electrons and protons, which have inherent discrete amounts of charge.\n\nIn simpler terms, quantisation implies that electric charge cannot be divided into smaller units indefinitely. It's similar to how money can only be divided into discrete amounts, such as pennies or cents, and you can't have a fraction of a penny or cent. In the provided context, electrons and protons have charges +e and –e, respectively, and these charges are intrinsic to these particles and cannot be altered.\n\nThe concept of quantisation is not limited to electric charge alone but is also applicable to other physical quantities, such as energy l

### Memory management:

In [25]:
from langchain.chains.conversation.memory import ConversationBufferMemory, ConversationKGMemory
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

In [26]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

In [27]:
llm = ChatCohere(cohere_api_key=cohere_api_key,
             temperature=0, 
             max_tokens = 256)

In [28]:
memory = ConversationBufferMemory()

In [29]:
new_memory = ConversationKGMemory(llm=llm)

In [30]:
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    # memory = memory, 
    memory=new_memory,
    prompt = prompt
)

In [31]:
conversation.predict(input="Hi there! I am Anurag")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi there! I am Anurag
AI:

> Finished chain.


"AI: Hello Anurag! It's a pleasure to meet you. I'm an AI assistant chatbot trained to assist and provide helpful and thorough responses. How can I help you today? Feel free to ask me anything, and I'll do my best to provide a detailed response based on the information I have."

In [36]:
conversation.predict(input="what is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: what is my name?
AI:

> Finished chain.


'I don\'t know your name, unfortunately. I can only provide information based on the "Relevant Information" section, and it doesn\'t contain any names. Is there anything else I can help you with?'

In [33]:
conversation.predict(input="Give me the information of Pune city")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Give me the information of Pune city
AI:

> Finished chain.


"Sure! Pune is a vibrant city located in the Indian state of Maharashtra. It is known for its pleasant climate, lush green surroundings, and rich cultural heritage. Here are some key facts and figures about Pune: \n\n- Population: Pune is Maharashtra's second-largest city and one of the major cities in India, with a population of over 3.1 million people as per the 2011 census. The city has been experiencing rapid urbanization and growth, with an estimated population of over 7 million in the Pune Metropolitan Region as of 2020. \n\n- Geography and Climate: Pune enjoys a unique geographical location, situated about 560 meters above sea level on the Deccan plateau. It is surrounded by the Sahyadri mountain range, providing a picturesque backdrop to the city. The city experiences a tropical wet and dry climate, with pleasant weather throughout the year. The average temperature during summers ranges from 20°C to 35°C, while winters are mild with temperatures rarely dipping below 10°C. \n\n-

In [34]:
conversation.predict(input="give me the landmarks and attractions only")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: give me the landmarks and attractions only
AI:


TooManyRequestsError: status_code: 429, body: data=None

In [ ]:
print(conversation.memory.chat_memory)

Human: Hi there! I am Anurag
AI: AI: Hello Anurag! It's a pleasure to meet you. I'm an AI assistant chatbot designed to provide helpful and thorough responses. How can I assist you today? Feel free to ask me anything, and I'll do my best to provide a detailed reply using the information I have access to.
Human: How are you today?
AI: I don't have feelings as an AI chatbot, but I'm always ready and eager to assist you with any queries or conversations you may have! How can I help you today?
Human: My TV is broken and I need some customer assistance
AI: AI: I'm sorry to hear that your TV is broken. I'd be happy to help you with customer assistance. Can you please provide me with some more details about the issue you're experiencing? Also, please note that to best help you, I will need the make and model of your TV.
Human: Give me the information of Pune city
AI: Sure! Pune is a vibrant city located in the Indian state of Maharashtra. It is known for its pleasant climate, lush green surro